In [ ]:
import pandas as pd 
import numpy as np 
import requests 

In [ ]:
class CriptoCoin:
    def __init__(self, ticker, start_date, end_date, period):
        self.ticker = ticker
        self.start_date = start_date+"T00:00:00"
        self.end_date = end_date+"T23:59:59"
        self.period = period 
        self.historical_data  =  CriptoCoin.downloadData(ticker, self.start_date, self.end_date, period)


    # download historical data
    @staticmethod    
    def downloadData(ticker, start_date,  end_date, period):
        granularity = {'M1': 60, 'M5': 300, 'M15':900, 'H1': 3600, 'H6': 21600, 'D1': 86400}
        # Mount URL
        url = f"https://api.exchange.coinbase.com/products/{ticker}/candles"
        # Make sure to pass dates as ISO
        params = { "start": start_date, "end": end_date, "granularity": granularity[period]}
        # Define request headers
        headers = {"content-type": "application/json"}
        # download historical data
        data = requests.get(url, params = params, headers=headers)
        columns = ['timestamp', "low", "high", "open", "close", "volume"]
        # Create a dataframe with results
        historical_data = pd.DataFrame(data=data.json() , columns=columns)
        # Create datetime column, set it as index and drop timestamp column
        historical_data['datetime'] = pd.to_datetime(historical_data['timestamp'], unit='s')
        historical_data.set_index('datetime', inplace=True)
        historical_data.drop('timestamp', axis = 1, inplace=True)
        return historical_data 


    def descritiveMetrics(self,  columns = 'full'):
        if columns == 'full':
            print(self.historical_data.describe())
        else:
            for column in columns:
                print(pd.DataFrame(self.historical_data[column].describe()))


    def percent_change(self, periods):
        output = self.historical_data.pct_change(periods = periods)
        return output.dropna() 


    def momoFactor(self, lag = 1,  columns = 'full'):
        s,  output, index  = lag, {}, self.historical_data.index 
    
        if columns == 'full':
            columns = self.historical_data.columns 
        
        for column in columns:
            momof, P = [], self.historical_data[column].values
            for t in range(s, len(P)):
                momof.append(np.log(P[t]/P[t - s]))
            output[column] = momof  
        
        output = pd.DataFrame(output, index =  index[s-1:len(index)-1])        
        return output 


    def mmFactor(self, lag = 1, columns = 'full'):
        s,  output, index  = lag, {}, self.historical_data.index 
        
        if columns == 'full':
            columns = self.historical_data.columns 
                
        for column in columns:
            mmf, P = [], self.historical_data[column].values
            for t in range(s, len(P)):
                mmf.append(sum(P[t-s:t])/len(P[t-s:t]))
            output[column] = mmf
                
        output = pd.DataFrame(output, index = index[s-1:len(index)-1])        
        return output    


    def smaFactor(self, lag = 1, columns = 'full'):
        s,  output, index  = lag, {}, self.historical_data.index 
            
        if columns == 'full':
            columns = self.historical_data.columns 
            
        for column in columns:
            smaf, P = [], self.historical_data[column].values
            for t in range(s, len(P)):
                mm =  sum(P[t-s:t])/len(P[t-s:t])
                smaf.append((P[t]/mm) - 1.0)
            output[column] = smaf  
            
        output = pd.DataFrame(output, index = index[s-1:len(index)-1])        
        return output 


    def smaRFactor(self, lag = 2, lag2 = 1, columns = 'full'):
            s, r,  output, index  = lag, lag2,  {}, self.historical_data.index 
                

            if columns == 'full':
                columns = self.historical_data.columns 
                
            for column in columns:
                smarf, P = [], self.historical_data[column].values
                for t in range(s, len(P)):
                    mms =  sum(P[t-s:t])/len(P[t-s:t])
                    mmr =  sum(P[t-r:t])/len(P[t-r:t])
                    smarf.append((mmr/mms) - 1.0)
                output[column] = smarf  
                
            output = pd.DataFrame(output, index = index[s-1:len(index)-1])        
            return output 

    def zscoreFactor(self, lag = 2, lag2 = 1, columns = 'full'):
            s, r,  output, index  = lag, lag2,  {}, self.historical_data.index 
                

            if columns == 'full':
                columns = self.historical_data.columns 
                
            for column in columns:
                smarf, P = [], self.historical_data[column].values
                for t in range(s, len(P)):
                    mms =  sum(P[t-s:t])/len(P[t-s:t])
                    sdts =  P[t-r:t]
                    smarf.append((mmr/mms) - 1.0)
                output[column] = smarf  
                
            output = pd.DataFrame(output, index = index[s-1:len(index)-1])        
            return output 


In [ ]:
bitcoin = CriptoCoin('BTC-USD', "2022-01-01", "2022-03-31", 'D1') 

In [ ]:
bitcoin.smaRFactor()

,low,high,open,close,volume
datetime,,,,,
2022-03-30,0.014537,0.000975,0.003918,0.016733,-0.158738
2022-03-29,0.000474,0.004256,-0.003187,0.003979,0.109114
2022-03-28,0.000786,0.001194,-0.003159,-0.003269,0.087113
2022-03-27,-0.024425,-0.013552,-0.025296,-0.003138,-0.289309
2022-03-26,-0.003910,-0.023204,-0.002273,-0.025283,-0.172747
...,...,...,...,...,...
2022-01-06,0.022426,0.007474,0.004213,0.017940,-0.113900
2022-01-05,0.000789,0.036118,0.026559,0.004072,0.093519
2022-01-04,0.034256,0.004823,0.006962,0.026650,-0.211350


In [ ]:
dados = bitcoin.historical_data 

saida = dados.rolling(0).mean().dropna() 
saida[:5]

,low,high,open,close,volume
datetime,,,,,
